In [1]:
import xarray as xr
import cartopy.crs as ccrs
import numpy as np
import time
import pandas as pd

In [3]:
pip install netCDF4

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 7.0/7.0 MB 39.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
# Ruta al archivo .nc
file_path = "fwi/1974.nc"  # Cambia esto al nombre del archivo

# Cargar el archivo NetCDF
dataset = xr.open_dataset(file_path)

# Ver información general del archivo
print(dataset)

<xarray.Dataset>
Dimensions:         (rlon: 424, rlat: 412, time: 365)
Coordinates:
  * rlon            (rlon) float64 -28.38 -28.27 -28.16 ... 17.94 18.05 18.16
  * rlat            (rlat) float64 -23.38 -23.27 -23.16 ... 21.62 21.73 21.84
  * time            (time) datetime64[ns] 1974-01-01 1974-01-02 ... 1974-12-31
    lon             (rlat, rlon) float64 ...
    lat             (rlat, rlon) float64 ...
Data variables:
    rotated_pole    |S1 ...
    fwi-daily-proj  (time, rlat, rlon) float64 ...
Attributes: (12/14)
    creation_date:  2021-06-25-T08:10:09Z
    Frequency:      Daily
    institution:    National Observatory of Athens, URL: http://www.noa.gr
    contact:        cgiannak@noa.gr
    title:          Daily FWI Values
    references:     van Wagner, C. E., (1987). Development and structure of a...
    ...             ...
    history:        Version: 2.00
    Lineage:        
    Summary:        
    Keywords:       
    License:        License to Use Copernicus Products
   

In [55]:
def obtener_fwi(dataset, fecha, lat_query, lon_query):
    """
    Obtiene el valor del Fire Weather Index (FWI) para una fecha y coordenadas específicas del dataset.
    
    Parámetros:
    dataset: xarray.Datasetm, contiene el FWI y coordenadas
    fecha: str o np.datetime64, fecha en formato 'YYYY-MM-DD'
    lat_query: float, latitud
    lon_query: float, longitud

    Retorna: float, el valor de FWI en las coordenadas especificadas para la fecha dada
    """
    
    # Seleccionar los datos del FWI para la fecha dada
    fwi_dia = dataset['fwi-daily-proj'].sel(time=fecha)
    
    # Obtenemos latitudes y longitudes
    latitudes = fwi_dia['lat'].values
    longitudes = fwi_dia['lon'].values

    # Inicializamos una variable que guarda la minima distancia 
    min_distance = float('inf')
    lat_index, lon_index = None, None

    # Recorremos las cuadriculas
    for i in range(latitudes.shape[0]):
        for j in range(longitudes.shape[1]):
            distance = np.sqrt((latitudes[i, j] - lat_query) ** 2 + (longitudes[i, j] - lon_query) ** 2)
            
            if distance < min_distance:
                min_distance = distance
                lat_index, lon_index = i, j

    # Si lo sindices son validos
    if lat_index is not None and lon_index is not None:
        fwi_value = fwi_dia.values[lat_index, lon_index]
        
        lat_value = latitudes[lat_index, lon_index]
        lon_value = longitudes[lat_index, lon_index]
        
        return fwi_value, lat_value, lon_value
    else:
        raise ValueError("No se encontraron valores")

In [4]:
bdif = pd.read_excel("bdif_geograficas.xlsx")
bdif.head()

,parte,año,cod_com,cod_prov,probignicion,diastormenta,diasultimalluvia,tempmaxima,humrelativa,velocidadviento,...,mes,tipodia,id_rel,provincia,poblacion,superficie,altitud,areaquemada,lon,lat
0,1974020249,1974,11,2,0.0,NaN,NaN,10.0,75.0,30.0,...,abril,laborable,1020864,Albacete,2478.0,51221.68,878.0,8.0,-2.318960,38.366800
1,1974020374,1974,11,2,0.0,NaN,NaN,NaN,NaN,NaN,...,mayo,festivo,1020371,Albacete,30516.0,77929.12,570.0,0.3,-1.703449,38.512190
2,1974020459,1974,11,46,0.0,NaN,NaN,NaN,NaN,NaN,...,junio,laborable,1460446,València/Valencia,5223.0,44668.00,596.0,0.6,-1.056085,39.059798
3,1974022274,1974,11,2,0.0,NaN,NaN,NaN,NaN,NaN,...,septiembre,laborable,1020674,Albacete,1326.0,8100.28,1126.0,2.0,-2.418393,38.499810
4,1974022457,1974,11,2,0.0,NaN,NaN,NaN,NaN,NaN,...,octubre,laborable,1020117,Albacete,601.0,14658.96,696.0,0.5,-2.070538,38.552177


In [56]:
# Ahora cargamos el archivo con el numero de parte, fecha y coordenadas
partes = bdif[['parte', 'deteccion', 'lon', 'lat']].sort_values(by='deteccion', ascending=True)
partes.head()

,parte,deteccion,lon,lat
2466,1974330044,1974-01-03 19:00:00,-5.663215,43.394746
1283,1974200015,1974-01-04 13:00:00,-1.948745,43.281484
245,1974080001,1974-01-04 16:00:00,1.767066,41.680295
1282,1974200014,1974-01-04 18:00:00,-1.948745,43.281484
2467,1974330045,1974-01-04 22:00:00,-6.414982,43.335256


In [57]:
# Filtro de fechas
inicio = pd.to_datetime('1988-01-01')
fin = pd.to_datetime('1988-12-31')

# Convertir la columna 'fecha' a formato datetime
partes['deteccion'] = pd.to_datetime(partes['deteccion'])

# Filtrar por rango
filtro = (partes['deteccion'] >= inicio) & (partes['deteccion'] <= fin)
partes = partes[filtro]
partes.head()

,parte,deteccion,lon,lat
98128,1988323215,1988-01-01 14:00:00,-7.610481,42.161122
98127,1988323214,1988-01-01 15:00:00,-7.650776,42.269964
98129,1988323216,1988-01-01 16:00:00,-7.610481,42.161122
98130,1988323217,1988-01-02 14:00:00,-7.650776,42.269964
98131,1988323218,1988-01-02 15:00:00,-7.650776,42.269964


In [58]:
fwi = []
start_time = time.time()

# Cargar el dataset
dataset = xr.open_dataset('fwi/1988.nc')

# Recorremos los incendios para obtener el FWI para cada uno
for _, parte in partes.iterrows():
    # Creamos el registro del incendio
    registro_fwi = {'parte': parte['parte']}
    
    # Cogemos la fecha, longitud y latitud
    fecha = pd.to_datetime(parte['deteccion']).strftime('%Y-%m-%d')
    lon = parte['lon']
    lat = parte['lat']

    # Obtener el valor del FWI
    try:
        fwi_value, lat_value, lon_value = obtener_fwi(dataset, fecha, lat, lon)
        registro_fwi['fwi'] = fwi_value
    
    except ValueError as e:
        print(f"Error al obtener FWI para el incendio {parte['parte']}: {e}")
        continue

    fwi.append(registro_fwi)

bdif_fwi = pd.DataFrame(fwi)

bdif_fwi.head()
bdif_fwi.to_excel("fwi/bdif_fwi_1988.xlsx", index=False)

end_time = time.time()
execution_time = (end_time - start_time)/60
print(f"Tiempo de ejecución: {execution_time} minutos")

Tiempo de ejecución: 51.05468399922053 minutos


### Fusión de archivos

In [59]:
# Listado de años
años = range(1974, 2021)

dataframes = []

# Vamos cargando los archivos
for año in años:
    # Montamos la ruta al archivo
    archivo = f'fwi/bdif_fwi_{año}.xlsx'
    
    df_año = pd.read_excel(archivo)
        
    dataframes.append(df_año)

bdif_fwi = pd.concat(dataframes, ignore_index=True)

# Mostrar el df
bdif_fwi.head()

,parte,fecha,lat,lon,fwi
0,1974330044,1974-01-03 19:00:00,43.394746,-5.663215,1.0
1,1974200015,1974-01-04 13:00:00,43.281484,-1.948745,1.0
2,1974080001,1974-01-04 16:00:00,41.680295,1.767066,1.0
3,1974200014,1974-01-04 18:00:00,43.281484,-1.948745,1.0
4,1974330045,1974-01-04 22:00:00,43.335256,-6.414982,1.0


In [61]:
bdif_fwi.to_excel('bdif_fwi.xlsx', index=False)
print(f"El archivo ha sido guardado como bdif_fwi")

El archivo ha sido guardado como bdif_fwi


In [62]:
# Cargar el dataset y mostrar las primeras filas
bdif_geo_clim = pd.read_excel('bdif_geo_clim.xlsx')
bdif_geo_clim.head()

,parte,año,cod_com,cod_prov,probignicion,diastormenta,diasultimalluvia,tempmaxima,humrelativa,velocidadviento,...,lon,lat,indicativo,prec,tmax,dir,velmedia,racha,sol,hrmedia
0,1974020249,1974,11,2,0.0,NaN,NaN,10.0,75.0,30.0,...,-2.318960,38.366800,8175,0.0,17.0,29.0,6.1,11.9,9.5,56.0
1,1974020374,1974,11,2,0.0,NaN,NaN,NaN,NaN,NaN,...,-1.703449,38.512190,8175,0.0,26.6,18.0,6.4,13.1,10.9,53.0
2,1974020459,1974,11,46,0.0,NaN,NaN,NaN,NaN,NaN,...,-1.056085,39.059798,8414A,0.0,23.0,9.0,3.6,11.1,3.3,71.0
3,1974022274,1974,11,2,0.0,NaN,NaN,NaN,NaN,NaN,...,-2.418393,38.499810,8175,0.0,28.0,NaN,5.0,NaN,8.9,63.0
4,1974022457,1974,11,2,0.0,NaN,NaN,NaN,NaN,NaN,...,-2.070538,38.552177,8175,0.0,23.2,NaN,4.7,NaN,9.3,45.0


In [63]:
# Unimos los dataframes
bdif_geo_clim_fwi = pd.merge(bdif_geo_clim, bdif_fwi, on='parte', how='left')

# Mostramos las primeras filas como comprobación
bdif_geo_clim_fwi.head()

,parte,año,cod_com,cod_prov,probignicion,diastormenta,diasultimalluvia,tempmaxima,humrelativa,velocidadviento,...,tmax,dir,velmedia,racha,sol,hrmedia,fecha,lat_y,lon_y,fwi
0,1974020249,1974,11,2,0.0,NaN,NaN,10.0,75.0,30.0,...,17.0,29.0,6.1,11.9,9.5,56.0,1974-04-25 17:00:00,38.366800,-2.318960,12.101361
1,1974020374,1974,11,2,0.0,NaN,NaN,NaN,NaN,NaN,...,26.6,18.0,6.4,13.1,10.9,53.0,1974-05-19 17:00:00,38.512190,-1.703449,69.039001
2,1974020459,1974,11,46,0.0,NaN,NaN,NaN,NaN,NaN,...,23.0,9.0,3.6,11.1,3.3,71.0,1974-06-11 10:00:00,39.059798,-1.056085,11.837668
3,1974022274,1974,11,2,0.0,NaN,NaN,NaN,NaN,NaN,...,28.0,NaN,5.0,NaN,8.9,63.0,1974-09-11 14:00:00,38.499810,-2.418393,42.358257
4,1974022457,1974,11,2,0.0,NaN,NaN,NaN,NaN,NaN,...,23.2,NaN,4.7,NaN,9.3,45.0,1974-10-01 08:00:00,38.552177,-2.070538,15.454229


In [64]:
bdif_geo_clim_fwi.to_excel('bdif_geo_clim_fwi.xlsx', index=False)
print(f"El archivo ha sido guardado como bdif_geo_clim_fwi")

El archivo ha sido guardado como bdif_geo_clim_fwi
